## 神经网络的学习算法实践

### 神经网络的学习步骤

主要有以下4个步骤：

- 选取输入数据：mini-batch从数据中随机选取一部分数据进行训练
- 计算梯度：计算各个权重参数的梯度
- 更新参数：将权重参数沿着梯度方向进行更新
- 重复上面的1-3步

上面的选择数据的过程是随机的，所以这里的梯度下降法有称为（Stochastic gradient descent）随机梯度下降法（SGD）：对随机选择的数据进行的梯度下降法


### 两层神经网络的实践

利用前面的学习的方法来实践手写数字识别的案例，前面一个章节中，我们提到利用训练好的数据参数来直接进行预测。下面我们通过这一节讲到的方法来实现参数的训练。

实践的类如下coding:


In [1]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient

class TowLayerNet:
    
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        # 初始化权重
        self.params = {}
        # 符合高斯分布的随机数进行初始化
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros(output_size)
        
    # 预测和推理函数    
    def predict(self,x):
        W1,W2 = self.params['W1'],self.params['W2']
        b1,b2 = self.params['b1'],self.params['b2']
        
        Z1 = np.dot(x,W1)+b1
        A1 = sigmoid(Z1)
        Z2 = np.dot(A1,W2)+b2
        y = softmax(Z2)
        
        return y
    
    # 损失函数
    def loss(self,x,t):
        y = self.predict(x)
        return cross_entropy_error(y,t)
    
    # 计算识别精度
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        
        accuracy = np.sum(y==t)/float(x.shape[0])
        return accuracy
    
    # 计算权重参数的梯度
    def numerical_gradient(self,x,t):
        # 定义损失函数
        loss_W  = lambda w: self.loss(x, t)
        
        # 计算相关的参数的偏导数
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    # 计算权重参数的梯度，高速版
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)

        return grads
        
        
        
        
        
    
    
        